# Two Components of Feature Engineering

 - 특징 추출(feature extraction)
     - 원시 데이터의 다양한 조작을 통해 모델 학습에 적합한 특징들을 생성
         - 조합: Ex. 부피 = 가로 X 세로 X 높이
         - 집계: 피보팅
         - 변환: binning, 파싱, 토그나이징, 인코딩, 로그변환, 스케일링 등
 
 - 특징 선택(feature selection)
     - 모델이 해결하고자 하는 문제와 상관없는 특징 = 노이즈
     - 전체 특징 집합 중에서 특정 모델 학습에 필요한 부분 집합을 선택
     

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)
print(df.head())

# Visualization setup
%matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode



                     CO(GT)  PT08.S1(CO)  PT08.S2(NMHC)  NOx(GT)  \
Datetime                                                           
2004-03-10 18:00:00     2.6       1360.0         1046.0    166.0   
2004-03-10 19:00:00     2.0       1292.0          955.0    103.0   
2004-03-10 20:00:00     2.2       1402.0          939.0    131.0   
2004-03-10 21:00:00     2.2       1376.0          948.0    172.0   
2004-03-10 22:00:00     1.6       1272.0          836.0    131.0   

                     PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)    RH  \
Datetime                                                                      
2004-03-10 18:00:00        1056.0    113.0        1692.0       1268.0  48.9   
2004-03-10 19:00:00        1174.0     92.0        1559.0        972.0  47.7   
2004-03-10 20:00:00        1140.0    114.0        1555.0       1074.0  54.0   
2004-03-10 21:00:00        1092.0    122.0        1584.0       1203.0  60.0   
2004-03-10 22:00:00        1205.0    116.0       

# Binning

 - 구간화(binning): 특정 변수를 범주형 변수로 변환
     - 목적: 강건한(robust) 모델 생성, 과적합(overwriting) 방지
     - 수치형/범주형 변수에 모두 적용 가능
         - 수치형 Ex. 소득 -> 소득 분위, 나이 -> 연령대
         - 범주형 Ex. 양산동, 세교동 -> 오산시, 진안동, 봉담읍 -> 화성시
 - 구간화 예시: 3학년 11반 학생들의 키
     - X = {155, 167, 173, 170, 171, 160, 275} -> 275는 잘못 입력한 값(이상치)
     - 평균 : 181.6cm
     - 이상치가 일정 수준 이상으로 측정될 경우 평균은 데이터를 대표하지 못함
     - 구간화 결과
         - x < 160 : 1명
         - 160 < x < 170 : 2명
         - 180 < x : 1명
         
         - 구간화를 통하여 얻어진 결과가 평균보다 데이터를 잘 설명함.

In [12]:
# Interpolate the 'CO(GT)' column
co = df['CO(GT)'].copy()
co.interpolate(inplace = True)

In [19]:
"""
Binning
"""

max_val = co.max()
min_val = co.min()
# print(max_val, min_val)

In [20]:
# Make interval values
bins = np.linspace(min_val, max_val, 6)

In [24]:
# labels for each bin
labels = ['0<=x<2.38', '2.38<=x<4.76', '4.76<=x<7.14','7.14<=x<9.52', '9.52<=x<11.9']

In [30]:
# Convert the numerical values into the categorical values
df['bins'] = pd.cut(co, bins = bins, labels = labels, include_lowest = True)

In [28]:
# print bins
df['bins']

Datetime
2004-03-10 18:00:00    2.38<=x<4.76
2004-03-10 19:00:00       0<=x<2.38
2004-03-10 20:00:00       0<=x<2.38
2004-03-10 21:00:00       0<=x<2.38
2004-03-10 22:00:00       0<=x<2.38
                           ...     
2005-04-04 10:00:00    2.38<=x<4.76
2005-04-04 11:00:00    2.38<=x<4.76
2005-04-04 12:00:00    2.38<=x<4.76
2005-04-04 13:00:00       0<=x<2.38
2005-04-04 14:00:00       0<=x<2.38
Name: bins, Length: 9357, dtype: category
Categories (5, object): [0<=x<2.38 < 2.38<=x<4.76 < 4.76<=x<7.14 < 7.14<=x<9.52 < 9.52<=x<11.9]

In [36]:
# Visualize the gistogram of bins
plt.hist(df['bins'], bins = 5)
plt.show()

# Log Transform

 - 로그 변환: 우편향된(right-skewed) 데이터 분포를 정규 분포에 가깝게 변환
     - 이상치에 강건한 모델 생성
     - 정규 분포에 적합한 알고리즘, 모수적 방법 적용이 용이해짐.

In [39]:
"""
Log Transform
"""

# Distribution of original data
# 0인 경우에 0 divide by zero 발생 -> log계산 시
# df.min()

sns.distplot(df['PT08.S3(NOx)'])

In [40]:
# Calculate natural Logarithm on 'PT08.S3(NOx)' column
df['log'] = np.log10(df['PT08.S3(NOx)'])

In [46]:
# Min values each columns
df.min()

CO(GT)                   0
PT08.S1(CO)            647
PT08.S2(NMHC)          383
NOx(GT)                  2
PT08.S3(NOx)           322
NO2(GT)                  2
PT08.S4(NO2)           551
PT08.S5(O3)            221
RH                     9.2
AH                  0.1847
C6H6(GT)                 0
bins             0<=x<2.38
log                2.50786
dtype: object

In [47]:
# Distribution after log transform
sns.distplot(df['log'])
plt.xlabel('log(NOx)')
plt.show()

# Encoding

 - 인코딩: 범주형 변수를 수치형 변수로 변환
     - 대부분의 모델/알고리즘은 수치형 데이털르 입력 받음.

In [73]:
"""
One-hot Encoding
"""
# make a dataset

emp_id = pd.Series([1, 2, 3, 4, 5])
gender = pd.Series(['Male', 'Female', 'Female', 'Male', 'Female'])
remarks = pd.Series(['Nice', 'Good', 'Great','Great','Nice'])

df_emp = pd.DataFrame()
df_emp['emp_id'] = emp_id
df_emp['gender'] = gender
df_emp['remarks'] = remarks

In [74]:
# Print unique values for each column

df_emp['emp_id'].unique()
df_emp['gender'].unique()
df_emp['remarks'].unique()

array(['Nice', 'Good', 'Great'], dtype=object)

In [75]:
# One-hot encoding the categorial values
df_emp_encoded = pd.get_dummies(df_emp, columns = ['gender', 'remarks'])
df_emp_encoded

,emp_id,gender_Female,gender_Male,remarks_Good,remarks_Great,remarks_Nice
0,1,0,1,0,0,1
1,2,1,0,1,0,0
2,3,1,0,0,1,0
3,4,0,1,0,1,0
4,5,1,0,0,0,1


# Label Encoding

 - 레이블 인코딩: 개별 범주를 특정 숫자 값(레이블)으로 표현
     - 순서형(ordinal) 데이터에 적용할 경우 데이터 순서, 순위 등을 보존 가능
    

# One-hot Encoding

 - 원핫 인코딩: 개별 범주를 특정 이진 백터(binary vector)로 표현
     - 범주 별로 인덱스를 할당
         - 범주에 해당되는 인덱스의 값: 1
         - 그 외의 나머지 값: 0
     - 명목형 데이터 인코딩에 적합

# Scaling

 - 일반적으로 변수들은 서로 다른 값 범위(스케일)를 가짐
     - Ex. 나이와 연소득
     - 모델은 기본적으로 변수 별 스케일을 고려하지 않음 -> 모델 학습 수렴 및 과적합 문제
 - 스케일링: 모든 변수들이 비슷한 값 범위를 갖도록 변환하는 작업
     - 정규화(normalization)
     - 표준화(standardization)

# Scaling: Normalization

 - 정규화: 모든 변수들을 0~1 사이의 값으로 스케일링
     - 대표적인 방법: min-max normalization(최대-최소 정규화)
     - 변수의 분포에는 변화가 없으므로, 이상치의 영향은 그대로 유지
     -> 정규화 전에 이상치 처리를 먼저 수행하는 것이 권장됨.

In [77]:
"""
Normalization
"""

# Visualize two columns of different scale
plt.plot(df['CO(GT)'], label = 'CO')
plt.plot(df['PT08.S3(NOx)'], label = 'NMHC')
plt.legend(loc = 'best')

In [78]:
# Normalize the 'CO(GT)' column

co = df['CO(GT)'].copy()
co_max = co.max()
co_min = co.min()

df['CO_Norm'] = (co - co_min) / (co_max - co_min)
df['CO_Norm']

Datetime
2004-03-10 18:00:00    0.218487
2004-03-10 19:00:00    0.168067
2004-03-10 20:00:00    0.184874
2004-03-10 21:00:00    0.184874
2004-03-10 22:00:00    0.134454
                         ...   
2005-04-04 10:00:00    0.260504
2005-04-04 11:00:00    0.201681
2005-04-04 12:00:00    0.201681
2005-04-04 13:00:00    0.176471
2005-04-04 14:00:00    0.184874
Name: CO_Norm, Length: 9357, dtype: float64

In [81]:
# Normalie the 'PT08.S2(NMHC)' column
nmhc = df['PT08.S2(NMHC)'].copy()
nmhc_max = nmhc.max()
nmhc_min = nmhc.min()

df['NMHC_Norm'] = (nmhc - nmhc_min) / (nmhc_max - nmhc_min)
df['NMHC_Norm']

Datetime
2004-03-10 18:00:00    0.362097
2004-03-10 19:00:00    0.312398
2004-03-10 20:00:00    0.303659
2004-03-10 21:00:00    0.308575
2004-03-10 22:00:00    0.247406
                         ...   
2005-04-04 10:00:00    0.392135
2005-04-04 11:00:00    0.351720
2005-04-04 12:00:00    0.371382
2005-04-04 13:00:00    0.315674
2005-04-04 14:00:00    0.362643
Name: NMHC_Norm, Length: 9357, dtype: float64

In [82]:
# Visualized normallized columns

plt.plot(df['CO_Norm'], label = 'CO (normalized)')
plt.plot(df['NMHC_Norm'], label = 'NMHC (normalized)')
plt.legend(loc = 'best')

# Scaling: Standardization

 - 표준화(또는 Z-score.정규화): 표준편차를 기반으로 스케일링 수행
     - 데이터가 정규분포를 따른다고 가정하고, 분포가 0을 중심으로 하고 표준편차가 1이 되도록 스케일링 수행
     - 변수마다 표준편차가 다를 경우 스케일링 결과(값 범위)가 다를 수 있음
     - 이상치에 영향을 덜 받는 스케일링 방법

# Scaling: Others

 - 그 외의 스케일링 기법
     - Max Abs Scaler
     - Robust Scaler
     - Quantile Tansformer Scaler
     - Power Tansformer Scaler
     - Unit Vector Scaler

# Grouping Operations

 - 깔끔한 데이터(tidy data): 데이터 분석 / 머신러닝에 적합한 데이터의 형태

# Grouping Operations: Pivot Table

 - 피벗 테이블: 개별 데이터 항목들의 집계 및 테이블 재구조화를 통해 데이터의 요약된(그룹호된) 결과를 나타내는 테이블
     - 트랜잭션 데이터(예: 계좌이체 내역)를 tidy 형태의 데이터로 변환할 때 활용

# feature Split

 - 특징 분할: 복합적인 값으로 구성된 특징 값을 여러개의 값으로 분할
     - Ex. 정제되지 않은 문자열의 토그나이징(tokenizing)
     